In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys



if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.2  Device :  cuda


In [2]:
import torchaudio
#import torchaudio.functional as F
import torchaudio.transforms as T

In [103]:
speaker_data = pd.read_excel("./voice_data/only_organics_healthy.xlsx")

In [104]:
speaker_data

,RECORDING,PATHOLOGY,DATE,SPEAKER,GENDER,AGE,DETAIL,DIAG
0,715,p,20.05.1998,1407,w,63,Laryngitis; Leukoplakie,structural
1,1303,p,21.04.1999,1407,w,64,Leukoplakie,structural
2,1557,p,08.12.1999,1407,w,65,Leukoplakie,structural
3,1559,p,15.12.1999,1407,w,65,Leukoplakie,structural
4,1864,p,29.11.2000,1407,w,66,Leukoplakie,structural
...,...,...,...,...,...,...,...,...
1461,80,n,13/01/1998,83,w,49,control,control
1462,81,n,13/01/1998,84,m,42,control,control
1463,82,n,13/01/1998,87,w,49,control,control
1464,83,n,13/01/1998,88,w,46,control,control


In [105]:
#파일과 일치하는 경우만 남긴다.
pathology = glob('D:/project/voice_pathology_ai/voice_data/organics/pathology/phrase/*.wav')
healthy = glob('D:/project/voice_pathology_ai/voice_data/organics/healthy/phrase/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))

pathology_record_files= [ path.split("\\")[-1] for path in pathology] # path 데이터 변환.
pathology_record_files = [ int(path.split("-")[0]) for path in pathology_record_files]

healthy_record_files= [ path.split("\\")[-1] for path in healthy] # path 데이터 변환.
healthy_record_files= [ int(path.split("-")[0]) for path in healthy_record_files] # path 데이터 변환.

Pathology :  597
Healthy:  634


In [106]:
speaker_data[speaker_data['PATHOLOGY']=='p']

,RECORDING,PATHOLOGY,DATE,SPEAKER,GENDER,AGE,DETAIL,DIAG
0,715,p,20.05.1998,1407,w,63,Laryngitis; Leukoplakie,structural
1,1303,p,21.04.1999,1407,w,64,Leukoplakie,structural
2,1557,p,08.12.1999,1407,w,65,Leukoplakie,structural
3,1559,p,15.12.1999,1407,w,65,Leukoplakie,structural
4,1864,p,29.11.2000,1407,w,66,Leukoplakie,structural
...,...,...,...,...,...,...,...,...
592,1314,p,28.04.1999,1397,m,77,Rekurrensparese,neurogenic
593,1564,p,15.12.1999,1397,m,78,Rekurrensparese,neurogenic
594,671,p,13.05.1998,1398,w,66,Rekurrensparese,neurogenic
595,879,p,15.07.1998,1398,w,66,Rekurrensparese,neurogenic


In [107]:
speaker_data[speaker_data['PATHOLOGY']=='n']

,RECORDING,PATHOLOGY,DATE,SPEAKER,GENDER,AGE,DETAIL,DIAG
597,1,n,20/11/1997,1,w,20,control,control
598,2,n,20/11/1997,2,w,22,control,control
599,3,n,20/11/1997,3,w,23,control,control
600,4,n,20/11/1997,4,m,22,control,control
601,5,n,20/11/1997,5,m,22,control,control
...,...,...,...,...,...,...,...,...
1461,80,n,13/01/1998,83,w,49,control,control
1462,81,n,13/01/1998,84,m,42,control,control
1463,82,n,13/01/1998,87,w,49,control,control
1464,83,n,13/01/1998,88,w,46,control,control


In [108]:
healthy_excel_list=speaker_data[speaker_data['PATHOLOGY']=='n'].RECORDING.tolist()

In [115]:
common_healthy = set(healthy_excel_list) & set(healthy_record_files)
common_healthy=list(common_healthy)

In [ ]:
isin_filter = (speaker_data['PATHOLOGY']=='n') & (speaker_data['RECORDING'].isin(common_healthy))
speaker_data[isin_filter]

In [120]:
speaker_available=pd.concat([ speaker_data[speaker_data['PATHOLOGY']=='p'],
            speaker_data[isin_filter]
          ],axis=0,ignore_index=True)

speaker_available

,RECORDING,PATHOLOGY,DATE,SPEAKER,GENDER,AGE,DETAIL,DIAG
0,715,p,20.05.1998,1407,w,63,Laryngitis; Leukoplakie,structural
1,1303,p,21.04.1999,1407,w,64,Leukoplakie,structural
2,1557,p,08.12.1999,1407,w,65,Leukoplakie,structural
3,1559,p,15.12.1999,1407,w,65,Leukoplakie,structural
4,1864,p,29.11.2000,1407,w,66,Leukoplakie,structural
...,...,...,...,...,...,...,...,...
1226,80,n,13/01/1998,83,w,49,control,control
1227,81,n,13/01/1998,84,m,42,control,control
1228,82,n,13/01/1998,87,w,49,control,control
1229,83,n,13/01/1998,88,w,46,control,control


In [123]:
speaker_available.to_excel("./voice_data/only_organics_healthy_available.xlsx",index=False)

----

In [148]:
speaker_data=pd.read_excel("./voice_data/only_organics_healthy_available.xlsx")

In [149]:
pathology = speaker_data[speaker_data['PATHOLOGY']=='p']['SPEAKER'].unique().tolist()
healthy = speaker_data[speaker_data['PATHOLOGY']=='n']['SPEAKER'].unique().tolist()
print(len(pathology))
print(len(healthy))

427
630


In [150]:
#겹치는 speaker는 곱하기 100을 해준다.
#3명이 겹친다.
changed_patients = list(set(healthy) & set(pathology))

for patient in changed_patients:
    temp=pathology[pathology.index(patient)]*100
    pathology[pathology.index(patient)] = temp
    

In [151]:
list(set(healthy) & set(pathology))

[]

In [152]:
#train test 나누기

import os
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split # train , test 분리에 사용.


X = pathology+healthy # path 데이터 합
print("총 데이터수 : ",len(X))
Y = [] # 라벨
for idx,x in enumerate(X):
    if idx<427:
        Y.append("pathology")
    else:
        Y.append("healthy")

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=456)
#stratify를 넣어서, test에도 라벨별 잘 분류되게 한다.

print("---")
print("훈련 셋 : ",len(Y),Counter(Y))
print("테스트 셋 : ",len(Y_test),Counter(Y_test))
print("---")



총 데이터수 :  1057
---
훈련 셋 :  845 Counter({'healthy': 504, 'pathology': 341})
테스트 셋 :  212 Counter({'healthy': 126, 'pathology': 86})
---


In [153]:
set(X) & set(X_test)

set()

In [154]:
#1. train, valid 나누기
#stratified kfold
import os
import random #데이터 shuffle 사용
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter



skf_iris = StratifiedKFold(n_splits=5,shuffle=True,random_state=456)
cnt_iter = 0

X_train_list = [] #데이터 셋 보관
Y_train_list = []

X_valid_list = []
Y_valid_list = []

for train_idx, test_idx in skf_iris.split(X,Y):
    
    #split으로 반환된 인덱스를 이용하여, 학습 검증용 테스트 데이터 추출
    cnt_iter += 1
    X_train, X_valid = [X[idx] for idx in train_idx.tolist() ], [X[idx] for idx in test_idx.tolist() ]
    Y_train, Y_valid = [Y[idx] for idx in train_idx.tolist() ], [Y[idx] for idx in test_idx.tolist() ]
    
    X_train_list.append(X_train)
    X_valid_list.append(X_valid)
    
    Y_train_list.append(Y_train)
    Y_valid_list.append(Y_valid)
    
    
    #학습 및 예측
    
    label_train = Y_train
    label_test = Y_valid
    unique_train, train_counts = np.unique(label_train, return_counts = True)
    unique_test, test_counts = np.unique(label_test, return_counts = True)
    
    uniq_cnt_train = dict(zip(unique_train, train_counts))
    uniq_cnt_test = dict(zip(unique_test, test_counts))
    
    
    
    print('교차 검증 : {}'.format(cnt_iter))
    print('학습 레이블 데이터 분포 : \n', uniq_cnt_train)
    print('검증 레이블 데이터 분포 : \n', uniq_cnt_test,'\n')



교차 검증 : 1
학습 레이블 데이터 분포 : 
 {'healthy': 403, 'pathology': 273}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 68} 

교차 검증 : 2
학습 레이블 데이터 분포 : 
 {'healthy': 403, 'pathology': 273}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 68} 

교차 검증 : 3
학습 레이블 데이터 분포 : 
 {'healthy': 403, 'pathology': 273}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 68} 

교차 검증 : 4
학습 레이블 데이터 분포 : 
 {'healthy': 403, 'pathology': 273}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 68} 

교차 검증 : 5
학습 레이블 데이터 분포 : 
 {'healthy': 404, 'pathology': 272}
검증 레이블 데이터 분포 : 
 {'healthy': 100, 'pathology': 69} 



In [155]:
set(X_train_list[0]) & set(X_test)

set()

In [157]:
# speaker to voice

all_train_record_list = []
all_valid_record_list = []
all_test_record_list = []

all_train_label_list = []
all_valid_label_list = []
all_test_label_list = []


#train
for fold_idx,fold in enumerate(X_train_list):
    #fold : speaker 번호들
    fold_record=[]
    fold_y_record=[]
    for idx,speaker_id in enumerate(fold):
        record_list = speaker_data[speaker_data['SPEAKER']==speaker_id]['RECORDING'].tolist()
        label_list = [ Y_train_list[fold_idx][idx] ] * len(record_list)
        
        fold_record += record_list
        fold_y_record += label_list
        
    all_train_record_list.append(fold_record)
    all_train_label_list.append(fold_y_record)

    
#valid
for fold_idx,fold in enumerate(X_valid_list):
    fold_record=[]
    fold_y_record=[]
    for idx,speaker_id in enumerate(fold):
        record_list = speaker_data[speaker_data['SPEAKER']==speaker_id]['RECORDING'].tolist()
        label_list = [ Y_valid_list[fold_idx][idx] ] * len(record_list)
        
        fold_record += record_list
        fold_y_record += label_list
        
    all_valid_record_list.append(fold_record)
    all_valid_label_list.append(fold_y_record)
    

#test
fold_record=[]
fold_y_record=[]
for idx,speaker_id in enumerate(X_test):
    record_list = speaker_data[speaker_data['SPEAKER']==speaker_id]['RECORDING'].tolist()
    label_list = [ Y_test[idx] ] * len(record_list)
    fold_record += record_list
    fold_y_record += label_list
all_test_record_list = fold_record
all_test_label_list = fold_y_record


X_train_list = all_train_record_list
X_valid_list = all_valid_record_list
X_test = all_test_record_list

Y_train_list = all_train_label_list
Y_valid_list = all_valid_label_list
Y_test = all_test_label_list



In [158]:
set(X_train_list[0]) & set(X_test)

set()

In [159]:
set(X_train_list[0]) & set(X_valid_list[0])

set()

In [161]:
#2. random over sampling
for i in range(5):
    X_temp = np.array(X_train_list[i]).reshape(-1,1)#각 데이터를 다 행으로 넣음. (1194,1)
    #Y = np.array(Y)
    ros = RandomOverSampler(random_state = 123)
    X_res,Y_res = ros.fit_resample(X_temp,Y_train_list[i])
    
    print("\n fold{} ".format(i))
    print('before dataset shape {}'.format(Counter(Y_train_list[i])) )
    print('Resampled dataset shape {}'.format(Counter(Y_res)) )   
    
    #원래대로 돌리기
    X_res=X_res.reshape(1, -1)
    X_train_list[i]=X_res[0].tolist()
    Y_train_list[i]=Y_res





 fold0 
before dataset shape Counter({'healthy': 406, 'pathology': 398})
Resampled dataset shape Counter({'healthy': 406, 'pathology': 406})

 fold1 
before dataset shape Counter({'healthy': 407, 'pathology': 398})
Resampled dataset shape Counter({'healthy': 407, 'pathology': 407})

 fold2 
before dataset shape Counter({'healthy': 408, 'pathology': 377})
Resampled dataset shape Counter({'healthy': 408, 'pathology': 408})

 fold3 
before dataset shape Counter({'healthy': 407, 'pathology': 375})
Resampled dataset shape Counter({'healthy': 407, 'pathology': 407})

 fold4 
before dataset shape Counter({'healthy': 408, 'pathology': 404})
Resampled dataset shape Counter({'healthy': 408, 'pathology': 408})


In [162]:
print('fold1 valid shape {}'.format(Counter(Y_valid_list[0])) )

fold1 valid shape Counter({'healthy': 103, 'pathology': 90})


In [163]:
print('testset shape {}'.format(Counter(Y_test)) )

testset shape Counter({'healthy': 128, 'pathology': 106})
